In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
# from google.colab import drive
# drive.mount('/content/gdrive')

In [2]:
val_data = pd.read_csv('val.csv')
train_data= pd.read_csv('train.csv')

val_data.head() # заполнить price нулями! 
                # сделать кластеризацию цен по категориям (проверить, сработает ли такое деление) 
                # можно сыграть на том, что исследуем отечественный рынок. Тогда при встрече латинских символов с какой-то вероятностью это - контакт
                # больше 4х цифр слитно, блять, или через пробел... аааа :(
                # добавляем просто флаги :) is_digit is_English(^A-z) и ещё помним про @ собАчку


,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Шины,Звонить 89425546881,Запчасти и аксессуары,Транспорт,2000.0,Тульская область,Огаревка,2019-10-10 00:00:25.200714,1
1,Продается мобильная перегородка с дверью,"Мобильная перегородка, предназначена для разгр...",Оборудование для бизнеса,Для бизнеса,10500.0,Вологодская область,Вологда,2019-10-10 00:03:11.527292,0
2,Комплект зимних шин на дисках 682/32/64,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,Запчасти и аксессуары,Транспорт,4000.0,Россия,Москва,2019-10-10 00:05:07.193248,1
3,Кровать-трансормер Дакота сб-4085,"Продаю кровать-трансформер производства ""Столп...",Мебель и интерьер,Для дома и дачи,17000.0,Московская область,Химки,2019-10-10 00:05:58.165179,0
4,Honda VFR 800 2004 г.в,"Мот в отличном состоянии для своих лет, Родной...",Мотоциклы и мототехника,Транспорт,235000.0,Брянская область,Брянск,2019-10-10 00:06:19.231151,0


In [ ]:
sub_cat = (val_data[val_data['is_bad']==0].subcategory.value_counts()/val_data.subcategory.value_counts()).sort_values(ascending = False)
val_data.subcategory.replace(sub_cat.keys(),sub_cat.values, inplace = True)

cat = (val_data[val_data['is_bad']==0].category.value_counts()/val_data.category.value_counts()).sort_values(ascending = False)
val_data['cat_freq'] = val_data.category.replace(cat.keys(),cat.values)
val_data

In [ ]:
sub_region = (val_data[val_data['is_bad']==0].region.value_counts()/val_data.region.value_counts()).sort_values(ascending = False)
val_data.region.replace(sub_region.keys(),sub_region.values, inplace = True)
val_data.region.fillna(0, inplace = True)

In [6]:
# val_data_2 = pd.read_csv('val.csv')
# scaler_price = StandardScaler()
# val_data_2.price.fillna(0, inplace=True)
# val_data_2['price_2']=scaler.fit_transform(val_data_2.price.to_numpy().reshape(-1,1))
# val_data_2['price_2'].value_counts()

In [7]:
# col = pd.get_dummies(val_data[['cat_freq','region','subcategory','text_feature_1']]) # Оставить как есть или попытаться перевести в One-hOt?
# col

# Предобработка всех объявлений

In [7]:
import re
import nltk # посмотрим, как они загружаются
nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('russian'))
#print(stopWords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Roman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# !pip install pymorphy2
# !pip install spacy==3.0
# !python -m spacy download ru_core_news_sm

In [8]:
from spacy.lang.ru import Russian  
import spacy 

from spacy.lang.ru.examples import sentences 
#import ru_core_news_sm
nlp = spacy.load('ru_core_news_sm')
from tqdm.notebook import tqdm

In [4]:
val_data

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
0,Шины,Звонить 89425546881,Запчасти и аксессуары,Транспорт,2000.0,Тульская область,Огаревка,2019-10-10 00:00:25.200714,1
1,Продается мобильная перегородка с дверью,"Мобильная перегородка, предназначена для разгр...",Оборудование для бизнеса,Для бизнеса,10500.0,Вологодская область,Вологда,2019-10-10 00:03:11.527292,0
2,Комплект зимних шин на дисках 682/32/64,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,Запчасти и аксессуары,Транспорт,4000.0,Россия,Москва,2019-10-10 00:05:07.193248,1
3,Кровать-трансормер Дакота сб-4085,"Продаю кровать-трансформер производства ""Столп...",Мебель и интерьер,Для дома и дачи,17000.0,Московская область,Химки,2019-10-10 00:05:58.165179,0
4,Honda VFR 800 2004 г.в,"Мот в отличном состоянии для своих лет, Родной...",Мотоциклы и мототехника,Транспорт,235000.0,Брянская область,Брянск,2019-10-10 00:06:19.231151,0
...,...,...,...,...,...,...,...,...,...
16232,Офисное помещение,Сдаются офисное помещение по Ул Калинина 80. П...,Коммерческая недвижимость,Недвижимость,9200.0,Чувашия,Чебоксары,2019-10-14 23:57:30.094904,0
16233,iPhone 8 Plus Silver 25GB,Оригинальный ростест айфон. Идеальное состояни...,Телефоны,Бытовая электроника,20000.0,Татарстан,Казань,2019-10-14 23:57:50.610616,0
16234,"6-к квартира, 54 м², 5/9 эт.",______________________________________________...,Квартиры,Недвижимость,3300000.0,Ставропольский край,Пятигорск,2019-10-14 23:58:02.781579,0
16235,"4-к квартира, 60 м², 9/9 эт.",/\n /\n● Работаем БЕЗ ПЕРЕРЫВОВ И ВЫХОДНЫХ с ...,Квартиры,Недвижимость,2300000.0,Ставропольский край,Пятигорск,2019-10-14 23:59:01.435691,0


In [224]:
def item_preprocess(texts):
    new_texts = []
    for doc in tqdm(nlp.pipe(texts, batch_size=1024, n_process=1, disable=["parser", "ner"])):
        new_texts.append(' '.join([tok.lemma_ for tok in doc if tok.lemma not in stopWords]))
        
    return new_texts

In [225]:
vectors_val = item_preprocess(val_data.description) # 10 минут 

In [18]:
%%time 
vectors = item_preprocess(train_data.description) # 7, ссуко,часов...

|          | 0/? [00:00<?, ?it/s]

Wall time: 7h 15min 37s


In [10]:
val_data.description[-1:].values

array(['Машина технически в отличном состояние все системы работают все исправно🛠🔩,машина ухоженная отличный салон💎ПТС оригинал 🇩🇪обслуживание в классике есть история много менял все новое оригинал все расскажу.хорошая комплектация мониторы💻🖥 ,шторки,проэкция,камеры и т.д есть 59 диски на липучке .Обмен желательно на джип ,налог не важен)Торг .за машину не стыдно приезжайте смотрите. Самое главное состояние👌Я хозяин отвечу на все вопросы...994 000-42-05'],
      dtype=object)

In [226]:
  # попробовать как предобработку так и постобработку
def easy_postproc(vectors):
    new_vectors = []
    for vector in vectors:
        vector = re.sub('[_,.!#;:]','',vector) # 
        vector = re.sub('[^А-я,^0-9,^A-z]',' ',vector) 
        new_vectors.append(vector)
    return new_vectors

In [243]:
# train_data['new_vectors'] = new_vectors
val_data['new_vectors'] = vectors_val
val_data.new_vectors.apply(remove_emoji)
val_data.new_vectors.apply(hard_postproc)
val_data['new_vectors'].values[-1]


0                                      звонить 89425546881
1        мобильный перегородка , предназначить для разг...
2        шина зимний б / у marshal assimetric i”zen kw ...
3        продавать кровать - трансформер производство "...
4        мот в отличный состояние для свой год , родной...
                               ...                        
16232    сдаваться офисный помещение по ул калинина 80 ...
16233    оригинальный ростест айфон . идеальный состоян...
16234    _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ ...
16235      / \n  / \n EMOJI работаем без перерывов и вы...
16236    машина технически в отличный состояние весь си...
Name: new_vectors, Length: 16237, dtype: object

0                                      звонить 89425546881
1        мобильный перегородка , предназначить для разг...
2        шина зимний б  у marshal assimetric i”zen kw 6...
3        продавать кровать  трансформер производство " ...
4        мот в отличный состояние для свой год , родной...
                               ...                        
16232    сдаваться офисный помещение по ул калинина 80 ...
16233    оригинальный ростест айфон  идеальный состояни...
16234                                                  ...
16235      \n   \n ● работаем без перерывов и выходных ...
16236    машина технически в отличный состояние весь си...
Name: new_vectors, Length: 16237, dtype: object

'машина технически в отличный состояние весь система работать все исправно 🛠 🔩 ,машина уходить отличный салон 💎 птс оригинал 🇩 🇪 обслуживание в классика быть история много менять весь новый оригинал весь расскажу.хорошая комплектация монитор 💻 🖥 , шторка , проэкция , камера и т.д быть 59 диск на липучка .обмен желательный на джип , налог не важен)торг .за машина не стыдно приезжать смотреть . самое главный состояние 👌 я хозяин ответить на весь вопрос ... 994 000 - 42 - 05'

# Веруем в нейросети!

In [14]:
from spacy.pipeline.textcat import DEFAULT_SINGLE_TEXTCAT_MODEL
nlp = spacy.blank("ru")
config = {
   "threshold": 0.5,
   "model": DEFAULT_SINGLE_TEXTCAT_MODEL,
}
textcat = nlp.add_pipe("textcat", config=config)

In [15]:
textcat.add_label("empty")
textcat.add_label("contact")

1

In [16]:
train_texts = val_data['new_vectors'].values[:12000]
train_labels = [{'cats': {'empty': label == 0,
                          'contact': label == 1}} 
                for label in val_data['is_bad']][:12000]

In [17]:
train_data = list(zip(train_texts, train_labels))
train_data[:3]

[('звонить 89425546881', {'cats': {'empty': False, 'contact': True}}),
 ('мобильный перегородка   предназначить для разграничение пространство в арендовать помещение   отлично подойти под ремонт мобильный техника   ремонт час   обувь и др услуг   перегородка можно собрать прямо или под угол 90 градус   угол можно сделать в любой место   длина 1500 высота 6500',
  {'cats': {'empty': True, 'contact': False}}),
 ('шина зимний б   у marshal assimetric i zen kw 61 681 41 r 63 диск ava   литой универсальный 4 j 63 3 600 или 3 663 2 ет 28 31   забрать можно на ярославский шоссе у мкад или в г   сергиев посад   мочь подвезти по договор нность   состояние  на 6 5 сезон   без торг и   8 964 688 36 68',
  {'cats': {'empty': False, 'contact': True}})]

In [26]:
from spacy.util import minibatch
from spacy.training.example import Example
import random 
spacy.util.fix_random_seed(1)
# optimizer = nlp.begin_training()

optimizer = nlp.initialize()
TRAIN_DATA = train_data
losses = {}
for epoch in tqdm(range(50)):
    random.shuffle(TRAIN_DATA)
    for batch in tqdm(minibatch(TRAIN_DATA, size=64)):
        for text, annotations in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update([example], drop=0.35, sgd=optimizer, losses=losses)
    print(losses)

  0%|          | 0/50 [00:00<?, ?it/s]

|          | 0/? [00:00<?, ?it/s]

{'textcat': 2770.9741192458987}


|          | 0/? [00:00<?, ?it/s]

{'textcat': 4628.207423436532}


...

{'textcat': 32629.121078720465}


|          | 0/? [00:00<?, ?it/s]

{'textcat': 33109.62744523918}


|          | 0/? [00:00<?, ?it/s]

{'textcat': 33554.14231971308}


In [45]:
texts =  val_data['new_vectors'].values[12000:]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores= textcat.predict(docs)


In [46]:
test_df =  val_data[12000:]
predictions = scores[:,1]
print(roc_auc_score(test_df.is_bad,predictions))

0.9256364819246412


In [53]:
test_df['scores'] = predictions
for i in test_df.category.unique():
    slice_ = test_df[test_df.category==i]
    print(i,roc_auc_score(slice_.is_bad,slice_.scores))

Услуги 0.9011863425925926
Транспорт 0.9582819978202151
Бытовая электроника 0.8356682769726247
Недвижимость 0.9411234546131391
Личные вещи 0.876103120005559
Для дома и дачи 0.9026118808727505
Работа 0.7177272727272728
Хобби и отдых 0.7721664275466283
Животные 0.846875
Для бизнеса 0.8339622641509434


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
def max_len(string):
    if string.split():
        return len(max(string.split(), key = len))
    else :
        return 0

def words_amount(string):
    return len(string.split())

def normalized_max(string):
    if string.split():
        return len(max(string.split(), key = len))/len(string)
    else:
        return 0
    
def  text_length(string):
    return  len(string)


def avg_word_lngth(string):
    average = lambda lst: sum(lst)/len(lst)
    avg = average([len(word) for word in string.split()])
    return avg


# |||||||||||||||||||||||||||||||||||

def hard_postproc(process):
    process = re.sub('/\n',' ',process.lower())
    process = re.sub('[-_—*#)(/•]','',process)
    process = re.sub('=','',process)
    process = re.sub('\\t','',process) 
    process = re.sub('\|','',process)
    process = re.sub('\.','',process) # над точкой подумать очень сильно!
    process = re.sub('мег', '925', process)
    process = re.sub('мтс', '925', process)
    process = re.sub('\\u202c', '', process)
    process = re.sub('\\u202d', '', process) 
    process  = re.sub('\\xa0', '', process)
    return process


def numbers_amount(string):
    return len(re.findall('\d',string))

def site_amount(string):
    key_words = 0
    keys = ['ru','com' ,'su' , 'рф', 'ру','net', 'www','cайт','http','org']
    for i in keys:
        key_words+=len(re.findall(i,string))
    return key_words

def messenger_amount(string):
    key_words = 0
    keys = ['what','вотс','ватс','вац','воц','вац','виб','вай','vib','tel','tg']
    for i in keys:
        key_words+=len(re.findall(i,string))
    return key_words

def social_amount(string):
    key_words = 0
    keys = ['инст','inst', 'вк','vk', 'ник','face','fb','ютуб','youtu']
    for i in keys:
        key_words+=len(re.findall(i,string))
    return key_words

def other_amount(string):
    key_words = 0
    keys = ['справ','услов','контакт','почт','связ']
    for i in keys:
        key_words+=len(re.findall(i,string))
    return key_words

    
def naive_detect(string):
    phones = re.findall('(?:\d{2,3}){4}',string)
    phone_length = 0
    for i in phones:
        phone_length+=len(i)
    return phone_length
                       
def english_letter_amount(string):
    letters = re.findall('[a-z]',string)
    letters_amount = 0
    for i in letters:
        letters_amount+=len(i)
    return letters_amount

def letter_number_amount(string):
    letter_numbers = 0
    word_numbers = ['сто','двести', 'триста', 'четыреста', 'пятьсот', 'шестьсот', 'семьсот','восемьсот','девятьсот'
            'двадцать', 'тридцать', 'сорок','пятьдесят','шестьдесят','семьдесят','восемьдесят', 'девяност', 
            'десять', 'одиннадцать', 'двенадцать','тринадцать','четырнадцать','пятнадцать','шестнадцать','семнадцать','восемнадцать', 'девятнадцать',
            'нул','нол','один', 'два', 'три','четыре','пят','шест', 'сем','восем','девят']                 
    for i in word_numbers:           
        letter_numbers+=len(re.findall(i,string))
    return letter_numbers
                            
def emoji_number_amount(string):
    emoji__words = 0
    emoji = ['0️⃣','1️⃣','2️⃣','3️⃣','4️⃣','6️⃣','5️⃣','7️⃣','8️⃣','9️⃣','৪','૭','Ƽ','૨','੫','੩','੫','Ӏ']
    for i in emoji:
        emoji__words+=len(re.findall(i,string))
    return emoji__words

def plus_coma(string):
    a = re.findall('@',string)
    b = 0
    for i in a:
        b+=len(i)
    return b
                       
# train_data['max_len'] = train_data.new_vectors.apply(max_len)
# train_data['words_amount'] = train_data.new_vectors.apply(words_amount)
# train_data['normalized_max'] = train_data.new_vectors.apply(normalized_max)
# train_data['text_length'] = train_data.new_vectors.apply(text_length)


def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                               
                            u"\U00002500-\U00002BEF"  # chinese char
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

                               
                               
                               
    return emoji_pattern.sub(r'EMOJI', text)

In [4]:
def create_features(val_data):
#     val_data['max_len'] = val_data.new_vectors.apply(max_len)
#     val_data['words_amount'] = val_data.new_vectors.apply(words_amount)
#     val_data['normalized_max'] = val_data.new_vectors.apply(normalized_max)
#     val_data['text_length'] = val_data.new_vectors.apply(text_length)
#     val_data['avg_word_lngth'] = val_data.new_vectors.apply(avg_word_lngth)

    val_data['hard'] = val_data.description.apply(hard_postproc)
    val_data['numbers_amount'] = val_data.hard.apply(numbers_amount)

    val_data['site_amount'] = val_data.hard.apply(site_amount)
    val_data['messenger_amount'] = val_data.hard.apply(messenger_amount)
    val_data['social_amount'] = val_data.hard.apply(social_amount)
    val_data['other_amount'] = val_data.hard.apply(other_amount)

    val_data['naive_detect'] = val_data.hard.apply(naive_detect)
    val_data['english_letter_amount'] = val_data.hard.apply(english_letter_amount)
    val_data['letter_number_amount'] = val_data.hard.apply(letter_number_amount)
    val_data['emoji_number_amount'] = val_data.hard.apply(emoji_number_amount)
    val_data['plus_coma'] = val_data.hard.apply(plus_coma)
    return val_data

In [12]:
# create_features(train_data)
train_data

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,hard,numbers_amount,site_amount,messenger_amount,social_amount,other_amount,naive_detect,english_letter_amount,letter_number_amount,emoji_number_amount,plus_coma
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - евро...,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0,продаем диванкровать удобный механизм еврокни...,6,1,0,1,1,0,0,0,0,0
1,Кожух рулевой колонки Даф хф 91 4509834,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0,кожух рулевой колонки daf xf 94 60066004 арт...,25,4,0,0,1,8,5,2,0,0
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1,! буст аккаунтов с ммр выше 1000ммр не беру ! ...,18,3,0,2,1,9,20,0,0,0
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1,продам телевизор диагональ 450наличие входа u...,3,1,0,0,0,0,39,1,0,0
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в ...,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0,открыткиконверты ручной работы выполнены в тех...,0,1,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984482,Штампы 61r,+79780979761/\nДенис/\nширину обода и вылет на...,Запчасти и аксессуары,Транспорт,2000.0,Крым,Мирное,2019-10-09 23:55:54.099019,1,+79780979761 денис ширину обода и вылет написа...,11,0,0,0,0,11,0,0,0,0
984483,Juul,Зарядка и под в комплекте.,Коллекционирование,Хобби и отдых,1200.0,Россия,Москва,2019-10-09 23:56:13.283059,0,зарядка и под в комплекте,0,0,0,0,0,0,0,0,0,0
984484,Белый стул IKEA,Белый стул Ikea /\nСостояние требует ремонта: ...,Мебель и интерьер,Для дома и дачи,500.0,Россия,Санкт-Петербург,2019-10-09 23:58:42.377241,0,белый стул ikea состояние требует ремонта: вн...,0,0,0,0,0,0,4,1,0,0
984485,"5-к квартира, 32 м², 2/2 эт.",Бронирование с октября по январь 1010 г./\n ...,Квартиры,Недвижимость,800.0,Калининградская область,Калининград,2019-10-09 23:59:26.744680,0,бронирование с октября по январь 1010 г !...,139,8,2,3,0,8,17,2,0,0


In [13]:

to_work = [
'Услуги',
'Бытовая электроника', 
'Хобби и отдых',
'Работа',
'Животные',
'Личные вещи']
train_data = train_data[train_data.category.isin(to_work)]
train_data[train_data.category =='Услуги'][train_data.is_bad==0].hard.values 

 


# (www.ysz72.ru)
# youtu.be/s3hZJMlZjoA'
# vk.com/dulis006
# vk . com /
#  Studio52. Ru',


# ВКонтакте Авто-подбор.РБ',
# Убрать как мусор  • (пробелом) \t



# # Попытка сделать числовую БИ? ТРИ? КВАТРО грамму? ! #считаем слитные цифры,
# # И, конечно, самые популярные МОНО_БИ_ТРИ_ГРАММЫ с определённой частотой


<ipython-input-13-ddcb68e4d4c1>:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_data[train_data.category =='Услуги'][train_data.is_bad==0].hard.values


array(['несколько вариантов разводки системы отопленияв целях экономии денежных средств изготовлю не сложный узел теплых полов быстрой настройки до десяти контуров замена старых труб на полипропилен в \\кухняваннабанягараж\\ установка водонагревателей умывальниковмоеккомпактовванндушевых кабинне дорого',
       '5       аттестация сотрудников:  рабочего персонала  более 500 направлений, охрана труда, работы на высоте, птм, электробезопасность, накс, лнк, в области пб, аттестация в области неразрушающего контроля, технология сварки, технология сварочного оборудования, электролаборатория 1       повышение квалификации: строительство, проектирование, строительный контроль, инженерные изыскания, мчс, экологическая безопасность 2       профессиональная переподготовка: охрана труда, строительство, проектирование, промышленная безопасность, мчс 4       оформление лицензий: мчс, фсб, минкультуры, экспертиза пб  готовы быть вашим представителем в ртн 6       оформление сертификации: исо 9005, 5

In [204]:
import re  #Если что вот второй номер: восемь-девять три восемь-пять ноль девять-три восемь-ноль ноль.
process =  'Выполняю различнын виды макияжа и причесок, а также художественное оформление бровей./\nБольше моих работ на странице инстаграм 📳 mariia_naumenko_makeup'
# process = 'В комнате два окна. /\nПо вопросам звонить МТС 5149262'.lower()
# Вся обработка  - после того, как напустили поиски по количеству цифр, цифробуквам,  цифросмайлам, наличию скобок
process = re.sub('/\n',' ',process.lower())
process = re.sub('[-_—*#)(/•]','',process)
process = re.sub('=','',process)
process = re.sub('\\t','',process) 
process = re.sub('\|','',process)
process = re.sub('\.','',process) # над точкой подумать очень сильно!
process = re.sub('мег', '925', process)
process = re.sub('мтс', '925', process)
process = re.sub('\\u202c', '', process)
process = re.sub('\\u202d', '', process) 
process  = re.sub('\\xa0', '', process)


# text = 'семммь'
# rep = re.compile(r'\b(\S*?)([а-я])\2{2,}\b')
# rep.sub(r'\1\2 ELONG', text)
a = re.findall('[@\+]',process)
b = 0
for i in a:
    b+=len(i)
b


process

0

'выполняю различнын виды макияжа и причесок, а также художественное оформление бровей больше моих работ на странице инстаграм 📳 mariianaumenkomakeup'

In [234]:
keywords_amount('звонить')

1

# Обучение модели


In [366]:

from art.defences.preprocessor  import LabelSmoothing
LS = LabelSmoothing()
new_x, new_y = LS(train_x, train_y)

ModuleNotFoundError: No module named 'art.defences'

In [369]:
!pip install adversarial-robustness-toolbox[all] 


SystemError: deallocated bytearray object has exported buffers
ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\cli\base_command.py", line 228, in _main
    status = self.run(options, args)
  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\cli\req_command.py", line 182, in wrapper
    return func(self, options, args)



  Using cached adversarial_robustness_toolbox-1.7.0-py3-none-any.whl (1.1 MB)
  Using cached numba-0.53.1-cp38-cp38-win_amd64.whl (2.3 MB)
  Using cached kornia-0.5.5-py2.py3-none-any.whl (292 kB)
  Using cached torchvision-0.10.0-cp38-cp38-win_amd64.whl (920 kB)
  Using cached mxnet-1.7.0.post2-py2.py3-none-win_amd64.whl (33.1 MB)
  Using cached tensorflow_addons-0.13.0-cp38-cp38-win_amd64.whl (615 kB)
  Using cached cma-3.1.0-py2.py3-none-any.whl (269 kB)
  Using cached opencv_python-4.5.2.54-cp38-cp38-win_amd64.whl (34.7 MB)
  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)


  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\commands\install.py", line 323, in run
    requirement_set = resolver.resolve(
  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\resolution\legacy\resolver.py", line 183, in resolve
    discovered_reqs.extend(self._resolve_one(requirement_set, req))
  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\resolution\legacy\resolver.py", line 388, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\resolution\legacy\resolver.py", line 340, in _get_abstract_dist_for
    abstract_dist = self.preparer.prepare_linked_requirement(req)
  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\operations\prepare.py", line 467, in prepare_linked_requirement
    local_file = unpack_url(
  File "C:\Users\Roman\anaconda3\lib\site-packages\pip\_internal\operations\prepare.py", line 255, in unpack_url
    file = ge

Found existing installation: llvmlite 0.34.0


ERROR: Cannot uninstall 'llvmlite'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [24]:
val_data['label'] = list(map(lambda x: 1 if (x==1) else -1, val_data.is_bad)) # !!!!!!!!!!!!!!!!!!!!!
#col['length'] = list(map(lambda x: len(x), val_data.new_vectors))
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = MinMaxScaler()


In [30]:
val_data.category.unique()

array(['Транспорт', 'Для бизнеса', 'Для дома и дачи', 'Личные вещи',
       'Услуги', 'Бытовая электроника', 'Недвижимость', 'Хобби и отдых',
       'Работа', 'Животные'], dtype=object)

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack , csr_matrix
from xgboost import XGBClassifier
from sklearn.model_selection import KFold  
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.compose import ColumnTransformer
from sklearn.model_selection  import ShuffleSplit
from sklearn.ensemble import RandomForestClassifier

In [18]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df = 0.4, min_df=0.001 )
vectorizer.fit(train_data.hard)

# clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, reg_alpha =0.2, reg_lambda =0.2 )
clf = RandomForestClassifier(n_jobs=-1, n_estimators = 500)
parameters = {'max_depth' : [5,10,15,20,25,30]}
grad_class  = GridSearchCV(clf, parameters, n_jobs=-1, verbose=1, scoring = 'roc_auc', cv  =5)
enc = DictVectorizer() 

for  i in tqdm( [ 'Личные вещи','Услуги', 'Бытовая электроника', 'Хобби и отдых',
       'Работа', 'Животные']):
    
    df_test = train_data[train_data.category == i] # 'max_len','words_amount','normalized_max','text_length', 

    categ_train = enc.fit_transform(df_test[['english_letter_amount','site_amount','messenger_amount','naive_detect','letter_number_amount','emoji_number_amount','plus_coma']].to_dict('records'))
    dense_vectors_train = vectorizer.transform(df_test.hard)
    X_train =hstack([dense_vectors_train,categ_train])#  
    y_train = df_test.is_bad.values
    grad_class.fit(X_train,y_train)
    print(i,' ',grad_class.best_params_, ' ', grad_class.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 43.8min finished


Личные вещи   {'max_depth': 30}   0.926016156113457
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 15.9min finished


Услуги   {'max_depth': 30}   0.9072930395930335
Fitting 5 folds for each of 6 candidates, totalling 30 fits


...

Хобби и отдых   {'max_depth': 30}   0.9097724049669674
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  6.1min finished


Работа   {'max_depth': 30}   0.8702404635817599
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  4.6min finished


Животные   {'max_depth': 30}   0.9463330404776414



Слова-инликаторы - телефон почта, пишите звоните
Label-smoothing 
Количество цифр 
Средняя длина числа  
Сколько чисел длины заданного типа 8_2_4_5_4_5_2_5_4_5 
Сколько 
**Регулярки переделать к хуям!**



In [23]:
val_data
val_data = val_data[val_data.category.isin(to_work)]
create_features(val_data)
# val_data[val_data.category =='Услуги'][val_data.is_bad==0].hard.values 
val_data
full_data = val_data.append(train_data)
full_data

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,hard,numbers_amount,site_amount,messenger_amount,social_amount,other_amount,naive_detect,english_letter_amount,letter_number_amount,emoji_number_amount,plus_coma
5,Селектив и Ниша Ценопад,Вычитаем 40 процентов 👍 Семейная коллекция пар...,Красота и здоровье,Личные вещи,500.0,Россия,Москва,2019-10-10 00:06:46.441433,0,вычитаем 40 процентов 👍 семейная коллекция пар...,78,5,1,1,0,0,1013,1,0,0
7,Лампа кольцевая,"Кольцевая лампа идеально подойдет, если вы: /\...",Красота и здоровье,Личные вещи,3200.0,Краснодарский край,Краснодар,2019-10-10 00:07:22.131219,0,"кольцевая лампа идеально подойдет, если вы: ...",26,0,0,3,0,0,10,2,0,0
8,Ремонт квартир,Бригада выполнит ремонт в вашей квартире любо...,Предложение услуг,Услуги,500.0,Московская область,Подольск,2019-10-10 00:08:30.768484,0,бригада выполнит ремонт в вашей квартире любо...,0,1,0,1,0,0,0,0,0,0
12,Coffee Lake i7-9700K \ GTX 2550Ti \ HDD 3000 GB,"👍 ГАРАНТИЯ, ТОВАРНЫЙ ЧЕК/\n 👨‍💻 ПРОВЕРИМ НА МЕ...",Настольные компьютеры,Бытовая электроника,6000.0,Россия,Санкт-Петербург,2019-10-10 00:14:27.600481,0,"👍 гарантия, товарный чек 👨‍💻 проверим на мест...",58,3,1,3,0,11,121,1,0,0
13,"33"" SAMSUNG 5180x5020 (51:50)","Samsung SyncMaster 4431NW/\nЖК-монитор, широко...",Товары для компьютера,Бытовая электроника,3000.0,Ростовская область,Азов,2019-10-10 00:15:17.380532,0,"samsung syncmaster 4431nw жкмонитор, широкофор...",39,2,0,0,0,0,36,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
984478,2D очки VR-BOX 6.0,В коробке с геймпадом Bluetooth./\nОчки универ...,Телефоны,Бытовая электроника,1000.0,Россия,Москва,2019-10-09 23:55:05.928422,0,в коробке с геймпадом bluetooth очки универсал...,7,0,0,0,0,0,10,0,0,0
984479,Одежда Barbie Monster High Ever After High,Вся информация про цены на фотографиях./\nЧерн...,Товары для детей и игрушки,Личные вещи,200.0,Россия,Санкт-Петербург,2019-10-09 23:55:25.699804,0,вся информация про цены на фотографиях черным\...,0,0,1,0,0,0,0,0,0,0
984481,Платье вечернее шёлк,"Покупала значительно дороже ! Шёлк , ручная ра...","Одежда, обувь, аксессуары",Личные вещи,5000.0,Россия,Москва,2019-10-09 23:55:47.714272,0,"покупала значительно дороже ! шёлк , ручная ра...",0,2,0,0,0,0,0,3,0,0
984483,Juul,Зарядка и под в комплекте.,Коллекционирование,Хобби и отдых,1200.0,Россия,Москва,2019-10-09 23:56:13.283059,0,зарядка и под в комплекте,0,0,0,0,0,0,0,0,0,0


In [24]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df = 0.4, min_df=0.001 )
vectorizer.fit(full_data.hard)

# clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, reg_alpha =0.2, reg_lambda =0.2 )
clf = RandomForestClassifier(n_jobs=-1, n_estimators = 500, max_depth=30)


TypeError: __init__() missing 1 required positional argument: 'param_grid'

In [27]:
parameters = {'max_depth' : [30]}
grad_class  = GridSearchCV(clf,param_grid = parameters, n_jobs=-1, verbose=1, scoring = 'roc_auc', cv  =5)
enc = DictVectorizer() 

for  i in tqdm( [ 'Личные вещи','Услуги', 'Бытовая электроника', 'Хобби и отдых',
       'Работа', 'Животные']):
    
    df_test = full_data[full_data.category == i] # 'max_len','words_amount','normalized_max','text_length', 

    categ_train = enc.fit_transform(df_test[['english_letter_amount','site_amount','messenger_amount','naive_detect','letter_number_amount','emoji_number_amount','plus_coma']].to_dict('records'))
    dense_vectors_train = vectorizer.transform(df_test.hard)
    X_train =hstack([dense_vectors_train,categ_train])#  
    y_train = df_test.is_bad.values
    grad_class.fit(X_train,y_train)
    print(i,' ',grad_class.best_params_, ' ', grad_class.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 15.8min finished


Личные вещи   {'max_depth': 30}   0.9249115740689782
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.4min finished


Услуги   {'max_depth': 30}   0.9050651343740347
Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Хобби и отдых   {'max_depth': 30}   0.9089297408789765
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.4min finished


Работа   {'max_depth': 30}   0.8709259675961546
Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


Животные   {'max_depth': 30}   0.9459619257689932



In [242]:

#grid = { 'penalty' : ('elasticnet','l1'), 'l1_ratio' : [0.1,0.2,0.3,0.4,0.45,0.5,0.6,0.7]}
cv = KFold(n_splits = 5, shuffle = True, random_state = 241) 
clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1 n_estimators = 500, max_depth=10)

weights = {0:1.0, 1:1.0}

Trans = ColumnTransformer(transformers=[
('tfidf', TfidfVectorizer(), 'new_vectors')])
pipeline = Pipeline([
    ('tfidf', Trans),
    ('clf', clf)
])



parameters = {
    'tfidf__tfidf__max_df': (0.4,  0.75), 
    'tfidf__tfidf__min_df': (None, 0.0005),
    'tfidf__tfidf__max_features': (None, 1000),
    'tfidf__tfidf__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__tfidf__use_idf': (True, False),
    'tfidf__tfidf__norm': ['l2'],
   
    
    'clf__max_depth': (None,10,20),
    'clf__scale_pos_weight' : [1,10,100],
    'clf__n_estimators': [500],
}

cv = ShuffleSplit(test_size=0.20, n_splits=1, random_state=0)
grad_class  = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring = 'roc_auc', cv  = cv)

for  i in tqdm( ['Услуги','Для дома и дачи', 'Бытовая электроника', 'Хобби и отдых',
       'Работа', 'Животные']):
    
    df_test = val_data[val_data.category == i]
    df_train = train_data[train_data.category == i]
    
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df = 0.8, min_df = 0.0005, max_features =1000)
    enc = DictVectorizer() 

    dense_vectors_train = vectorizer.fit_transform(df_train.new_vectors)
    dense_vectors_test = vectorizer.transform(df_test.new_vectors)
#     df_train_categ = enc.fit_transform(df[['price']].to_dict('records')) 
    
    categ_train = enc.fit_transform(df_train[['max_len','words_amount','normalized_max','text_length']].to_dict('records'))
    categ_test = enc.transform(df_test[['max_len','words_amount','normalized_max','text_length','avg_word_lngth']].to_dict('records'))
    
    X_train = hstack([dense_vectors_train,categ_train])
    X_test = hstack([dense_vectors_test,categ_test])

    y_train = df_train.is_bad.values
    y_test =  df_test.is_bad.values
    
    #   X_train, X_test, y_train, y_test= train_test_split(dense_vectors, df.label.values, test_size=0.2, random_state=0) # давай ещё вспомним про 1 и -1 вместо 0 и 1
#     grad_class.fit(df[['subcategory', 'cat_freq', 'max_len','words_amount','normalized_max','text_length', 'region','new_vectors']], df.label.values)

    #   grad_class_eval = grad_class.predict_proba(X_train[round(y_train.shape[0]*0.8):])[:,1]
    #   grad_class_pred = grad_class.predict_proba(X_test)[:,1]
    
#     grad_class.fit(df_train[['max_len','words_amount','normalized_max','text_length','new_vectors']],df_train.is_bad)
#     predictions =grad_class.predict_proba(df_test[['max_len','words_amount','normalized_max','text_length','new_vectors']])[:,1]
#     print(grad_class.best_params_, ' ',roc_auc_score(df_test.is_bad,predictions))
    clf.fit(X_train,y_train)
    predictions =  clf.predict_proba(X_test)[:,1]
    print(i, ' ',roc_auc_score(y_test,predictions))
  
 # c.append(roc_auc_score(y_test,grad_class_pred))
  # d.append(roc_auc_score(y_test,grad_boost_pred))
  # e.append(roc_auc_score(y_test,log_reg_pred))


AttributeError: 'DataFrame' object has no attribute 'new_vectors'

# Лучшие параметры обучения моделей


**Для бизнеса**
*{'clf__max_depth': 10, 'clf__n_estimators': 500, 'clf__scale_pos_weight': 1, 'tfidf__tfidf__max_df': 0.75, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': True}*    **0.9145274376745796 XGB**

**Для дома и дачи**
*{'clf__max_depth': 20, 'clf__n_estimators': 500, 'clf__scale_pos_weight': 1, 'tfidf__tfidf__max_df': 0.4, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': False}*   **0.9047643413700036 XGB**


**Транспорт**
*{'clf__l1_ratio': 0.2, 'clf__penalty': 'elasticnet', 'tfidf__max_df': 0.4, 'tfidf__max_features': None, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__use_idf': True}* **0.9598336933051392  SGD**

**Недвижимость**
*{'clf__l1_ratio': 0.2, 'clf__penalty': 'elasticnet', 'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 'tfidf__use_idf': True}*   **0.9052561525363874 SGD**

**'Услуги',**
{'clf__max_depth': 20, 'clf__n_estimators': 500, 'clf__scale_pos_weight': 1, 'tfidf__tfidf__max_df': 0.75, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 1), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': True}  ***0.8873351087992447** XGB*

**Бытовая электроника**
{'clf__max_depth': 20, 'clf__n_estimators': 500, 'clf__scale_pos_weight': 1, 'tfidf__tfidf__max_df': 0.75, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': False}   **0.9523663432267885** XGB
 

**Хобби и отдых',**
{'clf__max_depth': 10, 'clf__n_estimators': 500, 'clf__scale_pos_weight': 1, 'tfidf__tfidf__max_df': 0.4, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': False}   **0.880266903084317** XGB


**Работа**
{'clf__max_depth': 20, 'clf__n_estimators': 500, 'clf__scale_pos_weight': 1, 'tfidf__tfidf__max_df': 0.75, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': False}   **0.9002906190862395** XGB


**Животные**
Животные   {'max_depth': 5}
(ngram_range=(1, 3), max_df = 0.4, min_df = 0.01)
XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 100,scale_pos_weight = 1000)
['english_letter_amount','keywords_amount','naive_detect','letter_number_amount','emoji_number_amount','plus_coma']
***0.9145578231292517   XGB***



**'Личные вещи',**
{'clf__class_weight': {-1: 100, 1: 1}, 'clf__kernel': 'poly', 'tfidf__max_df': 0.6, 'tfidf__max_features': 10000, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__use_idf': True}   **0.9156026369950421** SVM SVC Проверить на метку класса при обучении




In [ ]:
from sklearn.model_selection import cross_val_score
test_vector = TfidfVectorizer(max_df =0.6, max_features = 1000, ngram_range = (1, 2), norm =  'l2', use_idf= True)
test_class = SGDClassifier(l1_ratio = 0.8, penalty = 'elasticnet')
df = val_data[val_data.category == 'Для бизнеса']
X_X = test_vector.fit_transform(df.new_vectors)
print(cross_val_score(test_class, X_X , df.label.values, cv=5,scoring = 'roc_auc'))

# Обучение и сохранение финальных моделей

In [16]:
from joblib import dump, load
from sklearn.pipeline import Pipeline

val_data['label'] = list(map(lambda x: 1 if (x==1) else -1, val_data.is_bad)) # !!!!!!!!!!!!!!!!!!!!!
SGD_train = val_data[val_data.category == 'Транспорт']
SGD_label = SGD_train.label
SGD_train = SGD_train.new_vectors


clf = SGDClassifier(l1_ratio= 0.2,penalty='elasticnet', loss = 'log')
params = {'tfidf__max_df': 0.4, 'tfidf__max_features': None,
 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__use_idf': True}
Trans = TfidfVectorizer(params)
transport  = Pipeline([
    ('tfidf', Trans),              # Вопрос - проактит ли то, что я обучил SGD на 0 и 1 
    ('clf', clf)])
transport.fit(SGD_train,SGD_label)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(input={'tfidf__max_df': 0.4,
                                        'tfidf__max_features': None,
                                        'tfidf__ngram_range': (1, 1),
                                        'tfidf__norm': 'l2',
                                        'tfidf__use_idf': True})),
                ('clf',
                 SGDClassifier(l1_ratio=0.2, loss='log',
                               penalty='elasticnet'))])

In [17]:
from joblib import dump, load
dump(transport, 'transport.joblib') 

['transport.joblib']

In [ ]:
{'clf__max_depth': 10, 'clf__n_estimators': 500, 'clf__scale_pos_weight': 1, 
0.927263811703171 XGB

In [96]:
XGB_train = train_data[train_data.category == 'Для бизнеса'] 
XGB_label = XGB_train.is_bad
XGB_train = XGB_train[['max_len','words_amount','normalized_max','text_length','new_vectors']]

params = {'tfidf__tfidf__max_df': 0.75, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 
             'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': True}
clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, max_depth=10, scale_pos_weight =1)
Trans = ColumnTransformer(transformers=[
('tfidf', TfidfVectorizer(params), 'new_vectors')])

business = Pipeline([
    ('tfidf', Trans),
    ('clf', clf)])

business.fit(XGB_train,XGB_label)

Pipeline(steps=[('tfidf',
                 ColumnTransformer(transformers=[('tfidf',
                                                  TfidfVectorizer(input={'tfidf__tfidf__max_df': 0.75,
                                                                         'tfidf__tfidf__max_features': None,
                                                                         'tfidf__tfidf__min_df': 0.0005,
                                                                         'tfidf__tfidf__ngram_range': (1,
                                                                                                       2),
                                                                         'tfidf__tfidf__norm': 'l2',
                                                                         'tfidf__tfidf__use_idf': True}),
                                                  'new_vectors')])),
                ('clf',
                 XGBClassifier(error='auc', max_depth=10, n_estimators=500,
         

In [97]:
dump(business, 'business.joblib')

['business.joblib']

In [98]:
# **Для дома и дачи**
 # **0.9318839611719654** XGB



XGB_train = train_data[train_data.category == 'Для дома и дачи'] 
XGB_label = XGB_train.is_bad
XGB_train = XGB_train[['max_len','words_amount','normalized_max','text_length','new_vectors']]

params = {'tfidf__tfidf__max_df': 0.4, 
 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 
 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': False} 

clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, max_depth=20, scale_pos_weight =1)
Trans = ColumnTransformer(transformers=[
('tfidf', TfidfVectorizer(params), 'new_vectors')])

Home_country = Pipeline([
    ('tfidf', Trans),
    ('clf', clf)])

Home_country.fit(XGB_train,XGB_label)
dump(Home_country, 'Home_country.joblib') 

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass input={'tfidf__tfidf__max_df': 0.4, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': False} as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


['Home_country.joblib']

In [99]:
# **'Услуги',**
#   ***0.8873351087992447** XGB*

XGB_train = train_data[train_data.category == 'Услуги'] 
XGB_label = XGB_train.is_bad
XGB_train = XGB_train[['max_len','words_amount','normalized_max','text_length','new_vectors']]

params = {'tfidf__tfidf__max_df': 0.6,
 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2),
 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': True}

clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, max_depth=10, scale_pos_weight =1)
Trans = ColumnTransformer(transformers=[
('tfidf', TfidfVectorizer(params), 'new_vectors')])

Services = Pipeline([
    ('tfidf', Trans),
    ('clf', clf)])

Services.fit(XGB_train,XGB_label)
dump(Services, 'Services.joblib') 

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass input={'tfidf__tfidf__max_df': 0.6, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': True} as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


['Services.joblib']

In [104]:
# **Бытовая электроника**


XGB_train = train_data[train_data.category == 'Бытовая электроника']
XGB_label = XGB_train.is_bad
XGB_train = XGB_train[['max_len','words_amount','normalized_max','text_length','new_vectors']]

params = {'tfidf__tfidf__max_df': 0.75,
 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 'tfidf__tfidf__ngram_range': (1, 2),
 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': False}  

clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, max_depth=20, scale_pos_weight =1)
Trans = ColumnTransformer(transformers=[
('tfidf', TfidfVectorizer(params), 'new_vectors')])

Electronics = Pipeline([
    ('tfidf', Trans),
    ('clf', clf)])

Electronics.fit(XGB_train,XGB_label)
dump(Services, 'Electronics.joblib') 

['Electronics.joblib']

In [105]:
# **Хобби и отдых',**

#                         0.880266903084317 XGB
                        
XGB_train = train_data[train_data.category == 'Хобби и отдых']
XGB_label = XGB_train.is_bad
XGB_train = XGB_train[['max_len','words_amount','normalized_max','text_length','new_vectors']]

params = {'tfidf__tfidf__max_df': 0.6, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005,
            'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': True}

clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, max_depth=10, scale_pos_weight =1)
Trans = ColumnTransformer(transformers=[
('tfidf', TfidfVectorizer(params), 'new_vectors')])

Hobby_chill = Pipeline([
    ('tfidf', Trans),
    ('clf', clf)])

Hobby_chill.fit(XGB_train,XGB_label)
dump(Hobby_chill, 'Hobby_chill.joblib')                         

['Hobby_chill.joblib']

In [106]:
# Работа 
#  0.9002906190862395 XGB

XGB_train = train_data[train_data.category == 'Работа']
XGB_label = XGB_train.is_bad
XGB_train = XGB_train[['max_len','words_amount','normalized_max','text_length','new_vectors']]

params = {'tfidf__tfidf__max_df': 0.6, 'tfidf__tfidf__max_features': None, 'tfidf__tfidf__min_df': 0.0005, 
        'tfidf__tfidf__ngram_range': (1, 2), 'tfidf__tfidf__norm': 'l2', 'tfidf__tfidf__use_idf': True}

clf = XGBClassifier(error = 'auc', objective = 'binary:logistic',  n_jobs=-1, n_estimators = 500, max_depth=10, scale_pos_weight =1)
Trans = ColumnTransformer(transformers=[
('tfidf', TfidfVectorizer(params), 'new_vectors')])

Work = Pipeline([
    ('tfidf', Trans),
    ('clf', clf)])

Work.fit(XGB_train,XGB_label)
dump(Work, 'Work.joblib')                         

['Work.joblib']

In [18]:
SGD_train = val_data[val_data.category == 'Личные вещи']
SGD_label = SGD_train.label
SGD_train = SGD_train.new_vectors


clf = svm.SVC(class_weight= {-1: 1, 1: 100}, kernel = 'poly', probability= True)
params =  {'tfidf__max_df': 0.6, 'tfidf__max_features': 10000, 'tfidf__ngram_range': (1, 1),
                'tfidf__norm': 'l2', 'tfidf__use_idf': True}
Trans = TfidfVectorizer(params)
personal_items  = Pipeline([
    ('tfidf', Trans),             
    ('clf', clf)])
personal_items.fit(SGD_train,SGD_label)
dump(personal_items, 'personal_items.joblib') 

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass input={'tfidf__max_df': 0.6, 'tfidf__max_features': 10000, 'tfidf__ngram_range': (1, 1), 'tfidf__norm': 'l2', 'tfidf__use_idf': True} as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


['personal_items.joblib']

In [21]:
# Недвижимость 
#  0.9221722594763471 SGD


SGD_train = val_data[val_data.category == 'Недвижимость']
SGD_label = SGD_train.label
SGD_train = SGD_train.new_vectors


clf = SGDClassifier(l1_ratio= 0.2,penalty='elasticnet', loss = 'log')

params = {'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 
 'tfidf__use_idf': True}
Trans = TfidfVectorizer(params)
realty  = Pipeline([
    ('tfidf', Trans),              # Вопрос - проактит ли то, что я обучил SGD на 0 и 1 
    ('clf', clf)])
realty.fit(SGD_train,SGD_label)
dump(realty, 'realty.joblib') 

D:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass input={'tfidf__max_df': 1.0, 'tfidf__max_features': None, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 'tfidf__use_idf': True} as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


['realty.joblib']

In [22]:
# Животные  0.8956916099773243 SGD

SGD_train = val_data[val_data.category == 'Животные']
SGD_label = SGD_train.label
SGD_train = SGD_train.new_vectors


clf = SGDClassifier(l1_ratio= 0.4,penalty='elasticnet', loss = 'log')

pets = {'tfidf__max_df': 0.75, 'tfidf__max_features': 50000, 'tfidf__ngram_range': (1, 2), 'tfidf__norm': 'l2', 
          'tfidf__use_idf': True}
Trans = TfidfVectorizer(params)
pets  = Pipeline([
    ('tfidf', Trans),              # Вопрос - проактит ли то, что я обучил SGD на 0 и 1 
    ('clf', clf)])
pets.fit(SGD_train,SGD_label)
dump(pets, 'pets.joblib') 

['pets.joblib']

In [26]:
train_again = train_data[train_data.category == 'Для дома и дачи']

In [27]:
model = load('Country_home.joblib')

In [ ]:

print(roc_auc_score())

In [43]:
# 
texts_only = [['Транспорт','transport.joblib'], ['Личные вещи','personal_items.joblib'],
['Недвижимость','realty.joblib'], ['Животные','pets.joblib']]
   



# Texts + numerical features 
texts_and_numerical = [['Для бизнеса','business.joblib' ],['Для дома и дачи','Country_home.joblib'],
                       ['Услуги','Services.joblib'],['Бытовая электроника','Electronics.joblib'],
                       ['Хобби и отдых','Hobby_chill.joblib'],['Работа','Work.joblib']]
    

In [63]:
a = []
for i in texts_only:
#     print(i[0], ' ',i[1])
    predict_data = val_data[val_data.category == i[0]]
    clf = load(i[1])
    a.append(clf.predict_proba(predict_data.new_vectors)[:,1])
print(a)


[array([0.95076377, 0.51839762, 0.30257122, ..., 0.70637687, 0.84072415,
       0.52449915]), array([0.04491916, 0.04485966, 0.97824228, ..., 0.04486807, 0.04479324,
       0.04488939]), array([0.7933067 , 0.01437309, 0.09831534, ..., 0.08926367, 0.00724316,
       0.02380986]), array([0.04385214, 0.92781776, 0.04081477, 0.95278805, 0.0274696 ,
       0.00794812, 0.93395797, 0.97123944, 0.0510493 , 0.00887528,
       0.0254976 , 0.06873381, 0.03328161, 0.05059038, 0.04689495,
       0.96300365, 0.93970814, 0.97211929, 0.94344968, 0.98959911,
       0.94218201, 0.01815059, 0.06173639, 0.97764895, 0.94269587,
       0.00578812, 0.94236874, 0.92281793, 0.9574739 , 0.96236304,
       0.9581291 , 0.99438125, 0.95313284, 0.95986514, 0.99790341,
       0.97236064, 0.04976973, 0.93255977, 0.10567994, 0.01668169,
       0.02960698, 0.01312015, 0.03397923, 0.07074584, 0.04209642,
       0.01867541, 0.98321565, 0.92763625, 0.01831381, 0.02089661,
       0.93208562, 0.03195666, 0.03773867, 0.03866

In [60]:
for i in texts_and_numerical:
#     print(i[0], ' ',i[1])
    predict_data = val_data[val_data.category == i[0]]
    clf = load(i[1])
    a.append(clf.predict_proba(predict_data[['max_len','words_amount','normalized_max','text_length','new_vectors']])[:,1])
print(a)


,max_len,words_amount,normalized_max,text_length
0,11,2,0.578947,19
1,13,40,0.045296,287
2,13,60,0.045775,284
3,14,96,0.019663,712
4,11,31,0.057292,192
...,...,...,...,...
16232,14,51,0.042683,328
16233,15,53,0.037313,402
16234,15,158,0.006903,2173
16235,15,170,0.007626,1967


In [1]:
!pip install jupyterthemes


In [2]:
!jt 

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl
